#### Prepare Annotations

```

annotation = {
'filename': img_path,
'width': w,
'height': h,
'ann': {
    'bboxes': bboxes.astype(np.float32),
    'labels': labels.astype(np.int64),
    'bboxes_ignore': bboxes_ignore.astype(np.float32),
    'labels_ignore': labels_ignore.astype(np.int64)
}
}

```

In [36]:
annotations = []

In [37]:
! head mmdetection_input.csv

cid,filename,width,height,xmin,ymin,xmax,ymax,num_tokens
122240045,500_F_122240045_e7QHZ8T7WLpsRWP3Ibx6quxaUKXQzs4T.jpg,500,392,29,29,90,50,token_1
209829899,500_F_209829899_lq6d3xclAVkA7XOu3GQKGvxgjqDBdjGF.jpg,500,375,14,14,90,34,token_2
13829594,500_F_13829594_PyEjWU070XPo7RCony9oi14UEi5LEr1T.jpg,500,334,351,14,482,49,token_10
86125167,500_F_86125167_zBSLCTXKbnPenEmP2C7vzyZtwPGkZaYO.jpg,500,334,367,260,481,303,token_5
38550534,500_F_38550534_8VArIDHCnuh8E8je08XTIRahTWeOOuYN.jpg,335,500,20,9,312,43,token_36
123720297,500_F_123720297_dxh673J3iAZVecM0HPbVEMKBfY4BXAN0.jpg,334,500,14,16,141,63,token_10
2754791,500_F_2754791_WgYr12CMnx7HQEfCMeJGYrfKcQD1K7.jpg,334,500,15,11,221,37,token_5
57345528,500_F_57345528_t9edGRRK9Uc0pezgyT34eSTG7JHIRS3B.jpg,500,383,9,7,189,27,token_5
184598300,500_F_184598300_T3DTjPD4RYTtQukmyz3QZzkSyjEyYBas.jpg,350,500,235,31,315,57,token_1


#### Split into test and train

In [50]:
import pandas as pd
import numpy as np

df = pd.read_csv('mmdetection_input.csv')


In [51]:
df['split'] = np.random.randn(df.shape[0], 1)

msk = np.random.rand(len(df)) <= 0.7

train = df[msk]
test = df[~msk]

In [52]:
train = train.drop(['split'], axis=1)
test = test.drop(['split'], axis=1)

In [53]:
train.to_csv('mmdetection_train.csv', index=None)

In [54]:
test.to_csv('mmdetection_val.csv', index=None)

In [55]:
! head -n 3 mmdetection_train.csv

cid,filename,width,height,xmin,ymin,xmax,ymax,num_tokens
122240045,500_F_122240045_e7QHZ8T7WLpsRWP3Ibx6quxaUKXQzs4T.jpg,500,392,29,29,90,50,token_1
13829594,500_F_13829594_PyEjWU070XPo7RCony9oi14UEi5LEr1T.jpg,500,334,351,14,482,49,token_10


In [56]:
#class_dict = {'1' : 1, '2' : 2, '3' : 3, '5' : 5, '10' : 10, '36' : 36}

def create_annotation(row_dict):
    annotation = {}
    annotation['filename'] = row_dict['filename']
    annotation['width'] = row_dict['width']
    annotation['height'] = row_dict['height']
    annotation['ann'] = {}
    annotation['ann']['bboxes'] = np.array([
                                             int(row_dict['xmin']),
                                             int(row_dict['ymin']),
                                             int(row_dict['xmax']),
                                             int(row_dict['ymax'])
                                            ])
    try:
        annotation['ann']['labels'] = row_dict['num_tokens']
    except:
        print('WTF')
    
    return annotation
                         

In [57]:
import csv
import numpy as np

line_number = 0

def csv_to_annotation_pkl(csv_file):
    annotations = []
    with open(csv_file) as f:
        reader = csv.reader(f, skipinitialspace=True)
        header = next(reader)
        while True:
            try:
                row = next(reader)
                line_number += 1
                row_dict = dict(zip(header, row))
                annotations.append(create_annotation(row_dict))
            except:
                break
    return annotations

In [58]:
train_annotations = csv_to_annotation_pkl('mmdetection_train.csv')

In [59]:
import mmcv
mmcv.dump(train_annotations, 'tp_train.pkl')

In [60]:
test_annotations = csv_to_annotation_pkl('mmdetection_val.csv')

In [61]:
mmcv.dump(test_annotations, 'tp_val.pkl')

### Move train and val datasets to another folder

In [64]:
import os
import shutil
src_file_name = '/home/ubuntu/text-placement/images/' + train.iloc[0]['filename']

True

In [68]:
def copy_file_to_folder(filename, dest_folder):
    src_file_name = '/home/ubuntu/text-placement/images/' + filename
    if os.path.exists(src_file_name):
        shutil.copy(src_file_name, dest_folder) 
    

In [69]:
copy_file_to_folder(train.iloc[0]['filename'], '/home/ubuntu/tp/mmdet-train')

In [72]:
train['filename'].apply(copy_file_to_folder, args=('/home/ubuntu/tp/mmdet-train',))

0        None
2        None
3        None
4        None
6        None
9        None
10       None
12       None
14       None
15       None
17       None
19       None
20       None
23       None
24       None
25       None
27       None
28       None
30       None
31       None
32       None
33       None
34       None
35       None
37       None
38       None
42       None
43       None
45       None
46       None
         ... 
13526    None
13527    None
13528    None
13530    None
13531    None
13532    None
13533    None
13534    None
13535    None
13537    None
13538    None
13539    None
13542    None
13543    None
13544    None
13545    None
13546    None
13547    None
13550    None
13551    None
13553    None
13554    None
13557    None
13558    None
13559    None
13563    None
13566    None
13567    None
13568    None
13569    None
Name: filename, Length: 9526, dtype: object

In [75]:
! ls -l /home/ubuntu/tp/mmdet-train/ | wc -l

6213


In [76]:
test['filename'].apply(copy_file_to_folder, args=('/home/ubuntu/tp/mmdet-val',))

1        None
5        None
7        None
8        None
11       None
13       None
16       None
18       None
21       None
22       None
26       None
29       None
36       None
39       None
40       None
41       None
44       None
47       None
50       None
53       None
59       None
61       None
68       None
69       None
72       None
77       None
79       None
82       None
84       None
87       None
         ... 
13483    None
13486    None
13490    None
13491    None
13495    None
13497    None
13498    None
13499    None
13503    None
13504    None
13511    None
13513    None
13514    None
13515    None
13521    None
13524    None
13529    None
13536    None
13540    None
13541    None
13548    None
13549    None
13552    None
13555    None
13556    None
13560    None
13561    None
13562    None
13564    None
13565    None
Name: filename, Length: 4044, dtype: object